<a href="https://colab.research.google.com/github/tibor/Twitter-Trends/blob/main/Twitter_Trends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install colab-env --upgrade

import requests
from requests_oauthlib import OAuth2
from collections import Counter
import os
import colab_env
from urllib.parse import quote
import pandas as pd
import time
from datetime import datetime
from dateutil.parser import parse

pd.set_option('display.max_colwidth', None)

base_url = 'https://api.twitter.com/1.1/search/tweets.json{}&tweet_mode=extended'

headers = {
    'authorization': os.getenv('TWITTER_OAUTH_2')
}

  Created wheel for colab-env: filename=colab_env-0.2.0-cp37-none-any.whl size=3837 sha256=5a9486a6eb7c077b21fb19574bb086f715ad328694fd652a003e2d9de37323b4
  Stored in directory: /root/.cache/pip/wheels/81/9c/85/7ec1faca43d1a5df32f33f4e22d5ae91f301a5406779528ead
Successfully built colab-env
Mounted at /content/gdrive


In [4]:
#@title Suchbegriff eingeben

text = "#laschetwusstees" #@param {type:"string"}
df = pd.DataFrame(columns=('ID', 'Retweets', 'Faves', 'Count', 'Text', 'Screen Name', 'Timestamp'))

next_results = '?q=' + quote( text ) + '%20exclude%3Anativeretweets'
print('Started searching..')
users = []
tweets = {}
tweets_dict = {}
oldest_tweet = datetime.now().strftime('%Y%m%d%H%M%S')

z = 0

while next_results:
  z += 1
  req = requests.get( base_url.format( next_results ), headers=headers ).json()
  #print( z )

  if 'statuses' in req:
    for status in req['statuses']:
      try:
        status_text = status['full_text']
      except:
        status_text = status['text']
      df.loc[len(df)] = [
        status['id'],
        status['retweet_count'],
        status['favorite_count'],
        status['retweet_count'] + status['favorite_count'],
        status_text,
        status['user']['screen_name'],
        status['created_at']
      ]
      oldest_tweet = status['created_at'] if parse(status['created_at']).strftime('%Y%m%d%H%M%S') < oldest_tweet else oldest_tweet
      id = status['id']
      tweets[id] = status['retweet_count']
      tweets_dict[id] = (status_text, status['user']['screen_name'], status['created_at'])
      users.append( status['user']['screen_name'] )
  if 'search_metadata' in req and 'next_results' in req['search_metadata']:
    next_results = req.get('search_metadata').get('next_results')
  elif 'errors' in req and req['errors'][0]['code'] == 88:
    print('API Limit reached. Collecting more tweets will start automatically in 15 minutes again. The oldest tweet yet is from {}'.format(oldest_tweet))
    time.sleep(900)
    print('API Limit should be free again. Script will continue to search for Tweets automatically')
  elif 'errors' in req:
    print(req)
  else:
    next_results = None
    print( 'Search ended succesfully' )

Started searching..
Search ended succesfully


In [6]:
df = df.sort_values('Retweets', ascending=False)

for x in range(10):
  print(
      '________\n',
      '{} Retweets: https://twitter.com/{}/status/{}'.format(
          df.iloc[x]['Retweets'],
          df.iloc[x]['Screen Name'],
          df.iloc[x]['ID']
      ),
      '\n',
      df.iloc[x]['Text']
  )

df = df.sort_values('Timestamp')
print( df.iloc[0] )

#sorted_tweets = sorted( tweets.items(), key=lambda x: x[1], reverse=True )
'''for i in range(50):
    print(
          '______________\n{} Retweets: https://twitter.com/{}/status/{}\n{}'.format(
                sorted_tweets[i][1],
                tweets_dict.get(sorted_tweets[i][0])[1],
                sorted_tweets[i][0],
                tweets_dict.get(sorted_tweets[i][0])[0]
          )
    )'''
users = Counter( users ).most_common()
for i in range(50):
  print( i, users[i] )

________
 228 Retweets: https://twitter.com/benjamin_stork/status/1417118820964225028 
 #Reul (CDU) gerade eben in der PK: „es ist nicht klug, immer alles vorher zu wissen, sondern, auf die Dinge zu reagieren, die passieren.“

Lasst euch das auf der Zunge zergehen. Die Toten waren nicht Unfall, sondern Konsequenz. Diese Partei muss weg. Komplett. #LaschetWussteEs
________
 212 Retweets: https://twitter.com/Gert_Woellmann/status/1417081244047876096 
 Warum trendet eigentlich #LaschetWussteEs, aber kein #DreyerWussteEs? Ahrweiler liegt z.B. in #RLP.
________
 198 Retweets: https://twitter.com/Leo_Proettel/status/1417056589866020864 
 #LaschetWussteEs. Die Landesregierung wurde gewarnt. Sie haben nicht zögerlich gehandelt, sondern gehofft, dass schon nichts passieren würde. Einfach ungeeignet für ein Amt mit dieser Verantwortung. #LaschetRuecktritt

https://t.co/KWBWlNfFS2
________
 175 Retweets: https://twitter.com/MartinSonneborn/status/1417128028283539456 
 lolSmiley!
#Laschet #Laschet